Để chạy các sổ tay sau, nếu bạn chưa làm, bạn cần triển khai một mô hình sử dụng `text-embedding-ada-002` làm mô hình cơ sở và đặt tên triển khai trong tệp .env là `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Tiếp theo, chúng ta sẽ tải Chỉ mục Nhúng vào một Dataframe của Pandas. Chỉ mục Nhúng được lưu trong một tệp JSON có tên là `embedding_index_3m.json`. Chỉ mục Nhúng chứa các Nhúng cho từng bản ghi phụ đề YouTube cho đến cuối tháng 10 năm 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Tiếp theo, chúng ta sẽ tạo một hàm gọi là `get_videos` để tìm kiếm trong Chỉ mục Embedding theo truy vấn. Hàm sẽ trả về 5 video hàng đầu có độ tương đồng cao nhất với truy vấn. Hàm hoạt động như sau:

1. Đầu tiên, một bản sao của Chỉ mục Embedding được tạo ra.
2. Tiếp theo, Embedding cho truy vấn được tính toán bằng cách sử dụng OpenAI Embedding API.
3. Sau đó, một cột mới được tạo trong Chỉ mục Embedding gọi là `similarity`. Cột `similarity` chứa độ tương đồng cosine giữa Embedding của truy vấn và Embedding cho mỗi đoạn video.
4. Tiếp theo, Chỉ mục Embedding được lọc theo cột `similarity`. Chỉ mục Embedding được lọc để chỉ bao gồm các video có độ tương đồng cosine lớn hơn hoặc bằng 0.75.
5. Cuối cùng, Chỉ mục Embedding được sắp xếp theo cột `similarity` và 5 video hàng đầu được trả về.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Hàm này rất đơn giản, nó chỉ in ra kết quả của truy vấn tìm kiếm.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Đầu tiên, Chỉ mục Nhúng được tải vào một Pandas Dataframe.  
2. Tiếp theo, người dùng được yêu cầu nhập một truy vấn.  
3. Sau đó, hàm `get_videos` được gọi để tìm kiếm Chỉ mục Nhúng theo truy vấn.  
4. Cuối cùng, hàm `display_results` được gọi để hiển thị kết quả cho người dùng.  
5. Người dùng sau đó được yêu cầu nhập một truy vấn khác. Quá trình này tiếp tục cho đến khi người dùng nhập `exit`.  

![](../../../../translated_images/vi/notebook-search.1e320b9c7fcbb0bc.webp)  

Bạn sẽ được yêu cầu nhập một truy vấn. Nhập truy vấn và nhấn enter. Ứng dụng sẽ trả về danh sách các video liên quan đến truy vấn. Ứng dụng cũng sẽ trả về một liên kết đến vị trí trong video nơi câu trả lời cho câu hỏi được đặt.  

Dưới đây là một số truy vấn để thử:  

- Azure Machine Learning là gì?  
- Mạng nơ-ron tích chập hoạt động như thế nào?  
- Mạng nơ-ron là gì?  
- Tôi có thể sử dụng Jupyter Notebooks với Azure Machine Learning không?  
- ONNX là gì?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Tuyên bố từ chối trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ gốc của nó nên được coi là nguồn chính xác và đáng tin cậy. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp do con người thực hiện. Chúng tôi không chịu trách nhiệm về bất kỳ sự hiểu lầm hoặc giải thích sai nào phát sinh từ việc sử dụng bản dịch này.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
